# Langchain

## Chain-of-Thought (CoT)

## Tree-of-Thought (ToT)

The Tree of Thought (ToT) is a chain that allows you to query a Large Language Model (LLM) using the Tree of Thought technique. 
This is based on the paper [\"Large Language Model Guided Tree-of-Thought\"](https://arxiv.org/pdf/2305.08291.pdf)"

In [7]:
import openai
import os

api_key = os.getenv("OPENAI_API_KEY")
organization = os.getenv("OPEN_AI_ORG")
# elevenlabs_key = os.getenv("ELEVENLABS_KEY")

In [9]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


template ="""
Step1 :
 
I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
"""

prompt = PromptTemplate(
    input_variables=["input","perfect_factors"],
    template = template                      
)

chain1 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt,
    output_key="solutions",
    verbose=True
)

In [10]:
template ="""
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:"""

prompt = PromptTemplate(
    input_variables=["solutions"],
    template = template                      
)

chain2 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt,
    output_key="review",
    verbose=True
)

In [11]:
template ="""
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:"""

prompt = PromptTemplate(
    input_variables=["review"],
    template = template                      
)

chain3 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt,
    output_key="deepen_thought_process",
    verbose=True
)


In [12]:
template ="""
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:"""

prompt = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template = template                      
)

chain4 = LLMChain(
    llm=ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        ),
    prompt=prompt,
    output_key="ranked_solutions",
    verbose=True
)

In [7]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)


reuslt = overall_chain({
        "input": "human colonization of Mars", 
        "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
    })




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Step1 :
 
I have a problem related to human colonization of Mars. Could you brainstorm three distinct solutions? Please consider a variety of factors such as The distance between Earth and Mars is very large, making regular resupply difficult
A:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

1. Sustainable Resource Management: One solution could be to focus on sustainable resource management on Mars. This would involve developing technologies and systems that allow for efficient use of available resources, such as water, energy, and food. By maximizing reso

In [8]:
reuslt.keys()

dict_keys(['input', 'perfect_factors', 'ranked_solutions'])

In [9]:
print(reuslt['ranked_solutions'])

Rank: 1

Justification: Vertical farming ranks first in terms of promise because it offers a sustainable and efficient method of food production on Mars. It maximizes land utilization and minimizes water usage, which are crucial factors in a resource-limited environment like Mars. Additionally, it reduces the need for importing food from Earth, making the colony more self-sufficient. The probability of success is high at 85%, and there is a high confidence level due to the extensive research and development already conducted in vertical farming on Earth.

Final thoughts and considerations: While there are challenges in terms of initial investment and implementation difficulty, these can be overcome with the right partnerships, resources, and strategies. Continuous monitoring and maintenance will be necessary, but with advancements in technology, this can be managed effectively. The limited variety of crops can be addressed by focusing on high-yield and nutrient-rich crops. Overall, ver

In [13]:
from langchain_experimental.tot.base import ToTChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_experimental.tot.checker import ToTChecker
from langchain.chat_models import ChatOpenAI

# import openai
import os

api_key = os.getenv("OPENAI_API_KEY")
organization = os.getenv("OPEN_AI_ORG")

llm = ChatOpenAI(
        temperature=0,
        openai_api_key=api_key,
        openai_organization=organization,
        )

checker = ToTChecker(
    inputs ={chain1, chain2, chain3, chain4},
    verbose=True
)

TypeError: Can't instantiate abstract class ToTChecker with abstract method evaluate

In [4]:
from langchain_experimental.tot.checker import ToTChecker
ToTChecker?

Init signature:
ToTChecker(
    *,
    memory: Optional[langchain.schema.memory.BaseMemory] = None,
    callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    output_key: str = 'validity',
) -> None
Docstring:     
Tree of Thought (ToT) checker.

This is an abstract ToT checker that must be implemented by the user. You
can implement a simple rule-based checker or a more sophisticated
neural network based classifier.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           /opt/homebrew/lib/python3.11/site-packages/langchain_experimental/tot/checker.py
Type:           ModelMetacl

In [ ]:
chain = ToTChain(
    llm = llm,
    memory = ConversationBufferWindowMemory(),
    checker = checker,
    verbose = True,
    verbose_llm = True
)

chain.run()

# Reference 
https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac
https://github.com/mrspiggot/forestOfThoughts/blob/master/langchain_monoculture.py